In [1]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys
import os

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [2]:
import numpy as np
from src.rocket import Rocket

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

# Rocket setup
Ts = 1 / 20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)


print("Linearization around a steady state:")

xs, us = rocket.trim()
print("Default trim function:")
print("xs = ", xs)
print("us = ", us)

xs, us = rocket.trim()
print("xs = ", xs)
print("us = ", us)

x_ref = np.array([0.0] * 9 + [1.0, 0.0, 3.0])
xs, us = rocket.trim(x_ref)
print("x_ref = ", x_ref)
print("xs = ", xs)
print("us = ", us)

Linearization around a steady state:

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Default trim function:
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
us =  [ 0.          0.         66.66666667  0.        ]
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
us =  [ 0.          0.         66.66666667  0.        ]
x_ref =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
us =  [ 0.          0.         66.66666667  0.        ]


## Todo 2.1: Trim and Linearize

### Understanding `trim()`
The trim function solves an optimization problem to find (xs, us) such that f(xs, us) ≈ 0.

For the rocket:
- **Default trim**: Hover at origin (all states = 0)
- **Custom trim**: Hover at specified reference position

Expected results:
- xs should have all velocities and angular velocities = 0
- us should have d1=0, d2=0, Pavg≈66.7%, Pdiff=0

# Part 2: Linearization

## Overview

In this part, we will:
1. **Trim** the system to find equilibrium (hover) state
2. **Linearize** the nonlinear dynamics around the equilibrium
3. **Decompose** into 4 independent subsystems
4. **Discretize** for use in discrete-time MPC

## Key Concepts

### Trimming
Finding steady-state (xs, us) where ẋ = f(xs, us) = 0 (equilibrium/hover)

### Linearization  
Approximating nonlinear dynamics around trim point:
```
ẋ ≈ A(x - xs) + B(u - us)
```
where A = ∂f/∂x and B = ∂f/∂u evaluated at (xs, us)

### Why Linearize?
- MPC with nonlinear models is computationally expensive
- Linear MPC has analytical solutions and guarantees
- Good approximation near trim point (small angles, velocities)

In [3]:
sys = rocket.linearize_sys(xs, us)
sys.info()  # see whole system


 System name: 'complete system'
------------------------------------------------------------
self.A shape: (12, 12)
self.B shape: (12, 4)
self.C shape: (12, 12)
self.D shape: (12, 4)
self.xs: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
self.us: [ 0.          0.         66.66666667  0.        ]

🔹 State variables:
        wx [rad/s]
        wy [rad/s]
        wz [rad/s]
     alpha [rad]
      beta [rad]
     gamma [rad]
        vx [m/s]
        vy [m/s]
        vz [m/s]
         x [m]
         y [m]
         z [m]
→ access via self.state_name, self.state_unit

🔹 Input variables:
        d1 [rad]
        d2 [rad]
      Pavg [%]
     Pdiff [%]
→ access via self.input_name, self.input_unit

🔹 Output variables:
        wx [rad/s]
        wy [rad/s]
        wz [rad/s]
     alpha [rad]
      beta [rad]
     gamma [rad]
        vx [m/s]
        vy [m/s]
        vz [m/s]
         x [m]
         y [m]
         z [m]
→ access via self.output_name, self.output_unit
-------------------------------------

### Understanding `linearize_sys()`

This function computes the Jacobian matrices A and B using CasADi automatic differentiation:
- **A** (12×12): How states evolve (∂f/∂x)
- **B** (12×4): How inputs affect state derivatives (∂f/∂u)

The result is an `LTISys` object containing:
- Matrices: A, B, C, D
- Trim point: xs, us  
- Metadata: state names, units, etc.

In [4]:
sys_x, sys_y, sys_z, sys_roll = sys.decompose()
sys_z.info()


 System name: 'sys_z'
------------------------------------------------------------
self.A shape: (2, 2)
self.B shape: (2, 1)
self.C shape: (1, 2)
self.D shape: (1, 1)
self.xs: [0. 3.]
self.us: [66.66666667]
self.idx: [ 8 11]
self.idu: [2]
self.idy: [11]

🔹 State variables:
        vz [m/s]
         z [m]
→ access via self.state_name, self.state_unit

🔹 Input variables:
      Pavg [%]
→ access via self.input_name, self.input_unit

🔹 Output variables:
         z [m]
→ access via self.output_name, self.output_unit
------------------------------------------------------------
 To check data directly, try:
   sys.A, sys.B, sys.C, sys.D
   sys.xs, sys.us, sys.state_name, etc.
------------------------------------------------------------


## Todo 2.2: Decompose into Independent Subsystems

The linearized 12-state system can be decomposed into **4 independent subsystems**:

| Subsystem | States | Input | Description |
|-----------|--------|-------|-------------|
| **sys_x** | ωy, β, vx, x | d2 | Pitch angle to x-position |
| **sys_y** | ωx, α, vy, y | d1 | Roll angle to y-position |
| **sys_z** | vz, z | Pavg | Throttle to height |
| **sys_roll** | ωz, γ | Pdiff | Differential throttle to yaw |

### Why are they independent?

At the trim point (upright hover), the coupling terms in the A and B matrices are zero or negligible because:
1. Small angle approximation: sin(α)≈0, cos(α)≈1 when α≈0
2. Symmetry: Rocket is symmetric about z-axis
3. Decoupled actuation: Each input primarily affects one set of states

In [5]:
print(sys_x.A, sys_x.B, sys_x.C, sys_x.D)
print(sys_y.A, sys_y.B, sys_y.C, sys_y.D)
print(sys_z.A, sys_z.B, sys_z.C, sys_z.D)
print(sys_roll.A, sys_roll.B, sys_roll.C, sys_roll.D)

[[0.    0.    0.    0.   ]
 [1.    0.    0.    0.   ]
 [0.    9.806 0.    0.   ]
 [0.    0.    1.    0.   ]] [[-65.47484472]
 [  0.        ]
 [  9.806     ]
 [  0.        ]] [[0. 0. 0. 1.]] [[0.]]
[[ 0.     0.     0.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.    -9.806  0.     0.   ]
 [ 0.     0.     1.     0.   ]] [[-65.47484472]
 [  0.        ]
 [ -9.806     ]
 [  0.        ]] [[0. 0. 0. 1.]] [[0.]]
[[0. 0.]
 [1. 0.]] [[0.14709]
 [0.     ]] [[0. 1.]] [[0.]]
[[0. 0.]
 [1. 0.]] [[0.104]
 [0.   ]] [[0. 1.]] [[0.]]


## Deliverable 2.1: Physical/Mechanical Explanation

**Question**: Explain why it's possible to separate this system into independent subsystems from an intuitive physical/mechanical perspective.

### Answer:

The rocket system naturally decouples into 4 independent subsystems at hover due to:

#### 1. **Geometric Symmetry**
- The rocket has cylindrical symmetry about the z-axis
- X and Y directions are equivalent → sys_x and sys_y have identical structure
- Z-axis (vertical) is fundamentally different → sys_z is independent

#### 2. **Decoupled Actuation**  
Each input controls a specific degree of freedom:
- **d1** (servo 1): Tilts rocket about x-axis → affects y-motion
- **d2** (servo 2): Tilts rocket about y-axis → affects x-motion  
- **Pavg** (average throttle): Controls vertical thrust → affects z-motion
- **Pdiff** (differential throttle): Creates torque about z-axis → affects yaw (γ)

#### 3. **Linearization at Hover**
At the trim point (upright, zero velocity):
- **No cross-coupling**: Tilting in x doesn't affect y (and vice versa) when angles are small
- **Gravity acts only in z**: Doesn't create moments about x or y axes when upright
- **Small angle approximation**: sin(θ) ≈ θ, cos(θ) ≈ 1 linearizes the trigonometric coupling

#### 4. **Physical Intuition**
Think of the rocket as 4 independent 1D problems:
- **Inverted pendulum in x**: Tilting rocket (β) creates horizontal acceleration in x
- **Inverted pendulum in y**: Tilting rocket (α) creates horizontal acceleration in y
- **Double integrator in z**: Throttle creates vertical acceleration
- **Spinning top about z**: Differential thrust creates yaw rotation

The subsystems only couple when:
- Angles become large (nonlinear regime)
- Rocket is not at hover (trim point changes)
- Multiple inputs are used simultaneously (cross-terms appear)

**Key insight**: At hover with small perturbations, the rocket's dynamics naturally separate into orthogonal degrees of freedom due to its physical symmetry and decoupled actuation.

In [6]:
Ad, Bd, Cd = sys_x._discretize(Ts)
print(Ad, Bd, Cd)

[[ 1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [ 5.00000000e-02  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.22575000e-02  4.90300000e-01  1.00000000e+00  0.00000000e+00]
 [ 2.04291667e-04  1.22575000e-02  5.00000000e-02  1.00000000e+00]] [[-3.27374224]
 [-0.08184356]
 [ 0.47692403]
 [ 0.0120903 ]] [[0. 0. 0. 1.]]


## Todo 2.3: Discretization

MPC operates in **discrete time** (sampled at intervals Ts = 1/20 = 0.05 seconds).

We need to convert continuous-time dynamics:
```
ẋ = Ax + Bu  (continuous)
```

To discrete-time dynamics:
```
x[k+1] = Ad·x[k] + Bd·u[k]  (discrete)
```

### Using `scipy.signal.cont2discrete`:
The function uses matrix exponential to compute exact discrete-time equivalent:
```python
Ad, Bd, Cd, _, _ = cont2discrete((A, B, C, D), dt=Ts, method='zoh')
```

### What 'zoh' means:
Zero-Order Hold assumes input u is constant between samples (realistic for digital controllers).

Let's discretize all 4 subsystems:

In [7]:
[Ad, Bd, Cd] = sys_y._discretize(Ts)
Ad, Bd, Cd

(array([[ 1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00],
        [ 5.00000000e-02,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.22575000e-02, -4.90300000e-01,  1.00000000e+00,
          0.00000000e+00],
        [-2.04291667e-04, -1.22575000e-02,  5.00000000e-02,
          1.00000000e+00]]),
 array([[-3.27374224],
        [-0.08184356],
        [-0.47692403],
        [-0.0120903 ]]),
 array([[0., 0., 0., 1.]]))

In [8]:
[Ad, Bd, Cd] = sys_z._discretize(Ts)
Ad, Bd, Cd

(array([[1.  , 0.  ],
        [0.05, 1.  ]]),
 array([[0.0073545 ],
        [0.00018386]]),
 array([[0., 1.]]))

In [9]:
[Ad, Bd, Cd] = sys_roll._discretize(Ts)
Ad, Bd, Cd

(array([[1.  , 0.  ],
        [0.05, 1.  ]]),
 array([[0.0052 ],
        [0.00013]]),
 array([[0., 1.]]))

## Summary of Part 2

### What We Accomplished:

1. ✓ **Trimmed** the system to find hover equilibrium
   - xs = all zeros (hover at origin)
   - us = [0, 0, 66.67, 0] (d1=0, d2=0, Pavg=66.67%, Pdiff=0)

2. ✓ **Linearized** around trim point
   - Obtained 12×12 A matrix and 12×4 B matrix
   - Linear approximation: ẋ ≈ A(x-xs) + B(u-us)

3. ✓ **Decomposed** into 4 independent subsystems
   - sys_x: (4 states) d2 → x position
   - sys_y: (4 states) d1 → y position  
   - sys_z: (2 states) Pavg → z position
   - sys_roll: (2 states) Pdiff → yaw angle

4. ✓ **Discretized** all subsystems
   - Sampling time: Ts = 0.05s (20 Hz)
   - Discrete dynamics: x[k+1] = Ad·x[k] + Bd·u[k]

### Key Insights:

- **Linearization is valid** near hover with small angles (< 10°)
- **Subsystems are decoupled** due to symmetry, small angles, and independent actuation
- **Discrete-time models** are ready for MPC controller design in Part 3

### Physical Understanding:

The rocket at hover behaves like **4 independent control problems**:
- **2 inverted pendulums** (x and y directions)
- **1 double integrator** (vertical z)  
- **1 rotational inertia** (yaw γ)

This decomposition **dramatically simplifies** controller design - we can design 4 small controllers instead of 1 large 12-state controller!

---

**Next**: Part 3 - Design MPC controllers for each subsystem